In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

In [4]:
# Cargar los datos
df = pd.read_csv('../../../sell-in.txt/sell-in.txt', sep='\t')
productosPredecir = pd.read_csv('C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/Labo3/datasets/Proyecto/Labo3/Datasets/productos_a_predecir.txt', sep='\t')


In [5]:

# Convertir el periodo a formato datetime
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')

# Agregar los datos por periodo y product_id para obtener la serie temporal
ts = df.groupby(['periodo', 'product_id'])['tn'].sum().reset_index()

# Asegurarse de que las columnas tengan el mismo tipo y formato
ts['product_id'] = ts['product_id'].astype(int)
ts['periodo'] = pd.to_datetime(ts['periodo'])

# Agregar lags a tus datos
lags = 3  # Número de lags a incluir
for lag in range(1, lags + 1):
    ts[f'tn_lag_{lag}'] = ts.groupby('product_id')['tn'].shift(lag)

# Eliminar filas con valores nulos resultantes de los lags
ts.dropna(inplace=True)

# Obtener la lista de productos únicos a predecir
product_ids = productosPredecir['product_id'].unique()

# Normalizar las series de tiempo (solo 'tn' y lags)
scaler_tn = StandardScaler()
scaler_lags = StandardScaler()

ts[['tn']] = scaler_tn.fit_transform(ts[['tn']])
ts[['tn_lag_1', 'tn_lag_2', 'tn_lag_3']] = scaler_lags.fit_transform(ts[['tn_lag_1', 'tn_lag_2', 'tn_lag_3']])

# Crear características adicionales si es necesario (Ejemplo: características temporales)
ts['year'] = ts['periodo'].dt.year
ts['month'] = ts['periodo'].dt.month

# Crear conjunto de entrenamiento y objetivo
X = ts[['product_id', 'year', 'month'] + [f'tn_lag_{lag}' for lag in range(1, lags + 1)]]  # Puedes agregar más características si están disponibles
y = ts['tn']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo LightGBM con Random Forest
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

params = {
    'objective': 'regression',
    'boosting': 'rf',  # Utilizar Random Forest para generar promedios en el árbol
    'num_iterations': 1000,  # Ajustar la cantidad de iteraciones según sea necesario
    'learning_rate': 0.003934720226268362,
    'num_leaves': 96,
    'bagging_freq': 7,  # Frecuencia de bagging
    'bagging_fraction': 0.6854002560591308,  # Fracción de bagging
    'verbose': -1
}

# Entrenamiento de LightGBM con barra de progreso
evals_result = {}
model = lgb.train(
    params, 
    train_data, 
    valid_sets=[train_data, test_data], 
    valid_names=['train', 'valid'],
    #evals_result=evals_result,
    #early_stopping_rounds=10, 
    #verbose_eval=10  # Muestra progreso cada 10 iteraciones
)

# Realizar predicciones para los productos a predecir
results = []
for product_id in tqdm(product_ids, desc="Predicting with LGBM"):
    product_data = ts[ts['product_id'] == product_id].copy()
    if not product_data.empty:
        # Predicción para el último periodo disponible + 1 mes
        last_period = product_data['periodo'].max()
        next_period = last_period + pd.DateOffset(months=1)
        next_data = pd.DataFrame({
            'product_id': [product_id],
            'year': [next_period.year],
            'month': [next_period.month],
            'tn_lag_1': [product_data['tn'].iloc[-1]],
            'tn_lag_2': [product_data['tn_lag_1'].iloc[-1]],
            'tn_lag_3': [product_data['tn_lag_2'].iloc[-1]]
        })
        
        pred = model.predict(next_data)
        results.append({
            'product_id': product_id,
            'predicted_tn': pred[0]
        })

# Convertir los resultados a un DataFrame
results_df = pd.DataFrame(results)

c:\Users\Josvaldes\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
Predicting with LGBM: 100%|██████████| 780/780 [00:02<00:00, 265.89it/s]


In [6]:
# Desnormalizar las predicciones solo de la columna 'predicted_tn'
results_df['predicted_tn'] = scaler_tn.inverse_transform(results_df[['predicted_tn']]).flatten()

In [7]:
# Ajustar predicciones negativas a cero
results_df['predicted_tn'] = results_df['predicted_tn'].apply(lambda x: max(x, 0))

# Asegurarse de que el DataFrame resultante tiene las columnas product_id y predicted_tn
results_df = results_df[['product_id', 'predicted_tn']]

# Exportar a un archivo CSV con las columnas product_id y predicted_tn
results_df.to_csv('C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/Labo3/datasets/Proyecto/Labo3/Predicciones/lgbm_predictionsv2.csv', index=False)

print("Predicciones exportadas a 'lgbm_predictionsv2.csv'")

# Evaluar el modelo
y_pred = model.predict(X_test)

# Calcular métricas
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absolute Error: {mae}')
print(f'R² Score: {r2}')


Predicciones exportadas a 'lgbm_predictionsv2.csv'
Root Mean Squared Error: 0.3158682921541013
Mean Absolute Error: 0.09488940242040152
R² Score: 0.8902636777451165


c:\Users\Josvaldes\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


borrrar...

In [13]:
# Cargar los datos
df = pd.read_csv('../../../sell-in.txt/sell-in.txt', sep='\t')
productosPredecir = pd.read_csv('C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/Labo3/datasets/Proyecto/Labo3/Datasets/productos_a_predecir.txt', sep='\t')

# Convertir el periodo a formato datetime
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')

# Agregar los datos por periodo y product_id para obtener la serie temporal
ts = df.groupby(['periodo', 'product_id'])['tn'].sum().reset_index()

# Asegurarse de que las columnas tengan el mismo tipo y formato
ts['product_id'] = ts['product_id'].astype(int)
ts['periodo'] = pd.to_datetime(ts['periodo'])

# Agregar lags a tus datos
lags = 3  # Número de lags a incluir
for lag in range(1, lags + 1):
    ts[f'tn_lag_{lag}'] = ts.groupby('product_id')['tn'].shift(lag)

# Eliminar filas con valores nulos resultantes de los lags
ts.dropna(inplace=True)

# Obtener la lista de productos únicos a predecir
product_ids = productosPredecir['product_id'].unique()

# Normalizar las series de tiempo (solo 'tn' y lags)
scaler = StandardScaler()
ts[['tn', 'tn_lag_1', 'tn_lag_2', 'tn_lag_3']] = scaler.fit_transform(ts[['tn', 'tn_lag_1', 'tn_lag_2', 'tn_lag_3']])

# Crear características adicionales si es necesario (Ejemplo: características temporales)
ts['year'] = ts['periodo'].dt.year
ts['month'] = ts['periodo'].dt.month

# Crear conjunto de entrenamiento y objetivo
X = ts[['product_id', 'year', 'month'] + [f'tn_lag_{lag}' for lag in range(1, lags + 1)]]  # Puedes agregar más características si están disponibles
y = ts['tn']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo LightGBM con Random Forest
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

params = {
    'objective': 'regression',
    'boosting': 'rf',  # Utilizar Random Forest para generar promedios en el árbol
    'num_iterations': 1000,  # Ajustar la cantidad de iteraciones según sea necesario
    'learning_rate': 0.003934720226268362,
    'num_leaves': 96,
    'bagging_freq': 7,  # Frecuencia de bagging
    'bagging_fraction': 0.6854002560591308,  # Fracción de bagging
    'verbose': -1
}

# Entrenamiento de LightGBM con barra de progreso
evals_result = {}
model = lgb.train(
    params, 
    train_data, 
    valid_sets=[train_data, test_data], 
    valid_names=['train', 'valid'],
    #evals_result=evals_result,
    #early_stopping_rounds=10, 
    #verbose_eval=10  # Muestra progreso cada 10 iteraciones
)

# Realizar predicciones para los productos a predecir
results = []
for product_id in tqdm(product_ids, desc="Predicting with LGBM"):
    product_data = ts[ts['product_id'] == product_id].copy()
    if not product_data.empty:
        # Predicción para el último periodo disponible + 1 mes
        last_period = product_data['periodo'].max()
        next_period = last_period + pd.DateOffset(months=1)
        next_data = pd.DataFrame({
            'product_id': [product_id],
            'year': [next_period.year],
            'month': [next_period.month],
            'tn_lag_1': [product_data['tn'].iloc[-1]],
            'tn_lag_2': [product_data['tn_lag_1'].iloc[-1]],
            'tn_lag_3': [product_data['tn_lag_2'].iloc[-1]]
        })
        
        pred = model.predict(next_data)
        results.append({
            'product_id': product_id,
            'predicted_tn': pred[0]
        })

# Convertir los resultados a un DataFrame
results_df = pd.DataFrame(results)

c:\Users\Josvaldes\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
Predicting with LGBM: 100%|██████████| 780/780 [00:04<00:00, 186.67it/s]


In [14]:
results_df

,product_id,predicted_tn
0,20001,10.324423
1,20002,10.324423
2,20003,10.324423
3,20004,5.985375
4,20005,4.822552
...,...,...
775,21263,-0.372676
776,21265,-0.372676
777,21266,-0.372676
778,21267,-0.372676


In [16]:
# Desnormalizar las predicciones solo de la columna 'predicted_tn'
results_df['predicted_tn'] = scaler.inverse_transform(results_df[['predicted_tn']]).flatten()

ValueError: non-broadcastable output operand with shape (780,1) doesn't match the broadcast shape (780,4)

In [15]:
# Desnormalizar las predicciones
results_df['predicted_tn'] = scaler.inverse_transform(results_df['predicted_tn'].values.reshape(-10, 10)).flatten()

ValueError: operands could not be broadcast together with shapes (78,10) (4,) (78,10) 

In [7]:
# Cargar los datos
df = pd.read_csv('../../../sell-in.txt/sell-in.txt', sep='\t')
productosPredecir = pd.read_csv('C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/Labo3/datasets/Proyecto/Labo3/Datasets/productos_a_predecir.txt', sep='\t')

# Convertir el periodo a formato datetime
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')

# Agregar los datos por periodo y product_id para obtener la serie temporal
ts = df.groupby(['periodo', 'product_id'])['tn'].sum().reset_index()

# Asegurarse de que las columnas tengan el mismo tipo y formato
ts['product_id'] = ts['product_id'].astype(int)
ts['periodo'] = pd.to_datetime(ts['periodo'])

# Agregar lags a tus datos
lags = 3  # Número de lags a incluir
for lag in range(1, lags + 1):
    ts[f'tn_lag_{lag}'] = ts.groupby('product_id')['tn'].shift(lag)

# Eliminar filas con valores nulos resultantes de los lags
ts.dropna(inplace=True)

# Obtener la lista de productos únicos a predecir
product_ids = productosPredecir['product_id'].unique()

# Normalizar las series de tiempo (excepto 'product_id' y 'periodo')
scaler = StandardScaler()
ts[['tn', 'tn_lag_1', 'tn_lag_2', 'tn_lag_3']] = scaler.fit_transform(ts[['tn', 'tn_lag_1', 'tn_lag_2', 'tn_lag_3']])

# Crear características adicionales si es necesario (Ejemplo: características temporales)
ts['year'] = ts['periodo'].dt.year
ts['month'] = ts['periodo'].dt.month

# Crear conjunto de entrenamiento y objetivo
X = ts[['product_id', 'year', 'month'] + [f'tn_lag_{lag}' for lag in range(1, lags + 1)]]  # Puedes agregar más características si están disponibles
y = ts['tn']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo LightGBM con Random Forest
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

params = {
    'objective': 'regression',
    'boosting': 'rf',  # Utilizar Random Forest para generar promedios en el árbol
    'num_iterations': 1000,  # Ajustar la cantidad de iteraciones según sea necesario
    'learning_rate': 0.003934720226268362,
    'num_leaves': 96,
    'bagging_freq': 7,  # Frecuencia de bagging
    'bagging_fraction': 0.6854002560591308,  # Fracción de bagging
    'verbose': -1
}

# Entrenamiento de LightGBM con barra de progreso
evals_result = {}
model = lgb.train(
    params, 
    train_data, 
    valid_sets=[train_data, test_data], 
    valid_names=['train', 'valid'],
    #evals_result=evals_result,
    #early_stopping_rounds=10, 
    #verbose_eval=10  # Muestra progreso cada 10 iteraciones
)

# Realizar predicciones para los productos a predecir
results = []
for product_id in tqdm(product_ids, desc="Predicting with LGBM"):
    product_data = ts[ts['product_id'] == product_id].copy()
    if not product_data.empty:
        # Predicción para el último periodo disponible + 1 mes
        last_period = product_data['periodo'].max()
        next_period = last_period + pd.DateOffset(months=1)
        next_data = pd.DataFrame({
            'product_id': [product_id],
            'year': [next_period.year],
            'month': [next_period.month],
            'tn_lag_1': [product_data['tn'].iloc[-1]],
            'tn_lag_2': [product_data['tn_lag_1'].iloc[-1]],
            'tn_lag_3': [product_data['tn_lag_2'].iloc[-1]]
        })
        
        pred = model.predict(next_data)
        results.append({
            'product_id': product_id,
            'predicted_tn': pred[0]
        })

# Convertir los resultados a un DataFrame
results_df = pd.DataFrame(results)



c:\Users\Josvaldes\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
Predicting with LGBM: 100%|██████████| 780/780 [00:03<00:00, 243.11it/s]


In [8]:
results_df

,product_id,predicted_tn
0,20001,10.324423
1,20002,10.324423
2,20003,10.324423
3,20004,5.985375
4,20005,4.822552
...,...,...
775,21263,-0.372676
776,21265,-0.372676
777,21266,-0.372676
778,21267,-0.372676


In [11]:
# Desnormalizar las predicciones
results_df['predicted_tn'] = scaler.inverse_transform(results_df['predicted_tn'].values.reshape(-10, 10)).flatten()

ValueError: operands could not be broadcast together with shapes (78,10) (4,) (78,10) 

In [ ]:
# Desnormalizar las predicciones
results_df['predicted_tn'] = scaler.inverse_transform(results_df['predicted_tn'].values.reshape(-1, 1)).flatten()

# Asegurarse de que el DataFrame resultante tiene las columnas product_id y predicted_tn
results_df = results_df[['product_id', 'predicted_tn']]

# Exportar a un archivo CSV con las columnas product_id y predicted_tn
results_df.to_csv('C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/Labo3/datasets/Proyecto/Labo3/Predicciones/lgbm_predictionsv2.csv', index=False)

print("Predicciones exportadas a 'lgbm_predictionsv2.csv'")

# Evaluar el modelo
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Root Mean Squared Error: {rmse}')